In [6]:
# Importing necessary libraries for training and visualization
import numpy as np
import pandas as pd
import torch
import torchvision
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
from torch.autograd import Variable
import tensorflow as tf
from sklearn import preprocessing

import matplotlib.pyplot as plt
import tqdm



train_set = pd.read_csv("/Users/cszilagyi2019/Desktop/All Coding/Titanic Predictions/train.csv")
test_set = pd.read_csv("/Users/cszilagyi2019/Desktop/All Coding/Titanic Predictions/test.csv")

In [7]:
#Setting up Training/Validation/Testing through the dataloaders

#Hyperparameters
batch_size = 1
validation_split = 0.2
num_epochs = 20

# Create validation split by taking a percentage of the training set:
dataset_size = len(train_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

train_indices, val_indices = indices[split:], indices[:split]

# Use the SubsetRandomSampler to randomly sample the training dataset for 
# training and validation data. We will feed this into the dataloader below.
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Create dataloader objects - will be used during training and inference
# to iterate over the data.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, \
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, \
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, \
                                          shuffle=False)

In [8]:
# Creating the Neural Network Class

class Net(nn.Module):

  def __init__(self, input_size, hidden_size, num_classes):

    super(Net, self).__init__()

    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.output = nn.Linear(hidden_size, num_classes)
    
    # Applying activation function
    self.sigmoid = nn.Sigmoid()

    
  #Forward pass
  def forward(self, x):

    out = self.fc1(x)
    out = self.sigmoid(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    out = self.output(out)
    #Will use cross entropy
    return out

In [47]:
#Making the dataset

class myDataSet(Dataset):
    
    def __init__(self, file):
        
        #Reading the file into variables
        self.x = file.iloc[:, [2,4,5,6,7,9,11]]
        self.y = file.iloc[1:892, 1]
        
        #Converting important string values to numbers so they can be fed into the nn
        self.x.replace('male', 0, inplace = True)
        self.x.replace('female', 1, inplace = True)
        self.x.replace('S', 0, inplace = True)
        self.x.replace('C', 1, inplace = True)
        self.x.replace('Q', 2, inplace = True)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        #Convert to tensor and return
        data = torch.from_numpy(self.x.values.astype(np.float32))
        return data[idx], self.y.iloc[idx]
        

In [48]:
#Setting up Training/Validation/Testing through the dataloaders

#Hyperparameters
batch_size = 1
validation_split = 0.2
num_epochs = 20

# Create validation split by taking a percentage of the training set:
dataset_size = len(train_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

train_indices, val_indices = indices[split:], indices[:split]

# Use the SubsetRandomSampler to randomly sample the training dataset for 
# training and validation data. We will feed this into the dataloader below.
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)


#Making the datasets
train_dataset = myDataSet(train_set)


# Create dataloader objects - will be used during training and inference
# to iterate over the data.
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, \
                                         sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, \
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, \
                                          shuffle=False)

In [49]:
#Making the neural network
net = Net(7, 10, 2)

#Cross Entropy Loss/Adam Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [53]:
#Training loop

running_loss = 0.0
dataiter = iter(train_loader)


for epoch in range(num_epochs):

  for i, (data, survived) in enumerate(train_loader):
    print(i)
    print(data)
    print(survived)
    
    # Clear the gradients before performing backpropagation
    optimizer.zero_grad()
    
    # Perform the forward pass - this call of net calls the forward() fn
    outputs = net(data)
    
    # Compute loss on the output of the forward pass and desired label
    loss = criterion(outputs, survived)
    # Compute the gradients with respect to the loss function
    loss.backward()      
    # Update the weights in the neural network using the optimizer/backprop
    optimizer.step()


0
tensor([[ 3.0000,  0.0000,     nan,  1.0000,  0.0000, 24.1500,  2.0000]])
tensor([0])
1
tensor([[ 3.0000,  0.0000, 21.0000,  0.0000,  0.0000,  7.2500,  0.0000]])
tensor([0])
2
tensor([[ 3.0000,  1.0000, 24.0000,  0.0000,  2.0000, 16.7000,  0.0000]])
tensor([0])
3
tensor([[ 2.0000,  0.0000,     nan,  0.0000,  0.0000, 15.0500,  1.0000]])
tensor([0])
4
tensor([[ 3.0000,  0.0000,     nan,  1.0000,  2.0000, 23.4500,  0.0000]])
tensor([0])
5
tensor([[ 3.0000,  1.0000,  2.0000,  0.0000,  1.0000, 10.4625,  0.0000]])
tensor([0])
6
tensor([[  1.0000,   0.0000,  17.0000,   0.0000,   2.0000, 110.8833,   1.0000]])
tensor([0])
7
tensor([[ 2.,  1., 42.,  0.,  0., 13.,  0.]])
tensor([1])
8
tensor([[ 2.,  0., 27.,  0.,  0., 26.,  0.]])
tensor([0])
9
tensor([[ 2.0000,  0.0000, 18.0000,  0.0000,  0.0000, 11.5000,  0.0000]])
tensor([0])
10
tensor([[ 3.0000,  1.0000, 25.0000,  0.0000,  0.0000,  7.7750,  0.0000]])
tensor([1])
11
tensor([[ 2.,  0., 34.,  1.,  0., 21.,  0.]])
tensor([0])
12
tensor([[ 3.0000

tensor([[3.0000, 0.0000,    nan, 0.0000, 0.0000, 8.7125, 1.0000]])
tensor([1])
107
tensor([[ 1.0000,  0.0000,     nan,  0.0000,  0.0000, 26.5500,  0.0000]])
tensor([1])
108
tensor([[3.0000, 0.0000,    nan, 0.0000, 0.0000, 7.2500, 0.0000]])
tensor([0])
109
tensor([[ 3.0000,  1.0000,     nan,  8.0000,  2.0000, 69.5500,  0.0000]])
tensor([0])
110
tensor([[ 2.,  1., 50.,  0.,  1., 26.,  0.]])
tensor([0])
111
tensor([[ 3.0000,  1.0000,  4.0000,  0.0000,  1.0000, 13.4167,  1.0000]])
tensor([1])
112
tensor([[ 3.0000,  0.0000, 25.0000,  0.0000,  0.0000,  7.7417,  2.0000]])
tensor([0])
113
tensor([[ 1.0000,  0.0000, 45.5000,  0.0000,  0.0000, 28.5000,  0.0000]])
tensor([0])
114
tensor([[ 3.0000,  1.0000, 13.0000,  0.0000,  0.0000,  7.2292,  1.0000]])
tensor([1])
115
tensor([[ 1.0000,  0.0000, 60.0000,  0.0000,  0.0000, 26.5500,  0.0000]])
tensor([0])
116
tensor([[ 3.0000,  0.0000, 19.0000,  0.0000,  0.0000,  7.6500,  0.0000]])
tensor([1])
117
tensor([[3.0000, 0.0000,    nan, 0.0000, 0.0000, 7.7

253
tensor([[3.0000, 0.0000,    nan, 0.0000, 0.0000, 8.0500, 0.0000]])
tensor([1])
254
tensor([[ 3.0000,  0.0000, 22.0000,  0.0000,  0.0000,  7.2500,  0.0000]])
tensor([0])
255
tensor([[ 3.0000,  1.0000,     nan,  1.0000,  0.0000, 15.5000,  2.0000]])
tensor([1])
256
tensor([[ 1.0000,  0.0000, 35.0000,  0.0000,  0.0000, 26.5500,  1.0000]])
tensor([0])
257
tensor([[ 3.0000,  1.0000, 39.0000,  0.0000,  5.0000, 29.1250,  2.0000]])
tensor([0])
258
tensor([[ 3.0000,  1.0000,  5.0000,  4.0000,  2.0000, 31.3875,  0.0000]])
tensor([0])
259
tensor([[ 3.0000,  0.0000, 15.0000,  1.0000,  1.0000,  7.2292,  1.0000]])
tensor([0])
260
tensor([[ 1.0000,  1.0000,     nan,  0.0000,  0.0000, 79.2000,  1.0000]])
tensor([1])
261
tensor([[ 3.0000,  0.0000, 19.0000,  0.0000,  0.0000, 10.1708,  0.0000]])
tensor([0])
262
tensor([[ 2.0000,  1.0000, 13.0000,  0.0000,  1.0000, 19.5000,  0.0000]])
tensor([1])
263
tensor([[ 3.0000,  0.0000, 11.0000,  0.0000,  0.0000, 18.7875,  1.0000]])
tensor([0])
264
tensor([[ 3.0

349
tensor([[ 3.0000,  0.0000, 25.0000,  1.0000,  0.0000,  7.7750,  0.0000]])
tensor([1])
350
tensor([[ 2.,  0., 25.,  1.,  0., 26.,  0.]])
tensor([0])
351
tensor([[ 2.0000,  1.0000,  7.0000,  0.0000,  2.0000, 26.2500,  0.0000]])
tensor([0])
352
tensor([[ 3.0000,  0.0000, 40.5000,  0.0000,  0.0000,  7.7500,  2.0000]])
tensor([1])
353
tensor([[ 2.0000,  0.0000, 29.0000,  0.0000,  0.0000, 10.5000,  0.0000]])
tensor([0])
354
tensor([[ 1.0000,  1.0000, 63.0000,  1.0000,  0.0000, 77.9583,  0.0000]])
tensor([0])
355
tensor([[ 1.0000,  1.0000, 60.0000,  1.0000,  0.0000, 75.2500,  1.0000]])
tensor([1])
356
tensor([[ 3.0000,  0.0000, 36.0000,  0.0000,  0.0000,  7.4958,  0.0000]])
tensor([1])
357
tensor([[ 3.0000,  1.0000, 18.0000,  0.0000,  0.0000,  6.7500,  2.0000]])
tensor([0])
358
tensor([[ 3.0000,  0.0000, 25.0000,  0.0000,  0.0000,  7.2250,  1.0000]])
tensor([0])
359
tensor([[ 3.0000,  1.0000, 21.0000,  0.0000,  0.0000,  7.7500,  2.0000]])
tensor([0])
360
tensor([[ 1.,  0., 31.,  1.,  0., 

IndexError: single positional indexer is out-of-bounds

In [39]:
# Function for getting accuracy, adapted from: 
# https://towardsdatascience.com/a-simple-starter-guide-to-build-a-neural-network-3c2cf07b8d7c

def get_accuracy(loader, my_net):
  correct = 0
  total = 0
  for images, labels in loader:
      images = Variable(images.view(-1, 28*28))
      outputs = my_net(images)
      _, predicted = torch.max(outputs.data, 1)  
      total += labels.size(0)               
      correct += (predicted == labels).sum()

  return 100 * correct / total